In [ ]:
# openai python csomag betöltése
from openai import OpenAI
# chat kliens létrehozása
client = OpenAI()

# chat üzenetek elküldése és válasz fogadása
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Egy versíró művész vagy, aki hétköznapi helyzetek alapján ír színes, elgondolkodtató verseket."},
    {"role": "user", "content": "Írj egy verset a hétvégi korán kelés fájdalmáról."}
  ]
)

print(completion.choices[0].message.content)

In [ ]:
from openai import OpenAI
client = OpenAI()

# üzenetek egymásutánjának küldése és válasz
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
response.model_dump()


In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" }, # <-- új mező
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020? Answer with year and winner fields."} # <-- megmondjuk mik a fő mezők
  ]
)
print(response.choices[0].message.content)
import json
my_dict = json.loads(response.choices[0].message.content)
print(my_dict)
print(my_dict['winner'])
print(my_dict['year'])

In [ ]:
from openai import OpenAI
client = OpenAI()

# Példa "CV"
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

# Prompt amiben leírjuk mit szeretnénk kinyerni a szövegből
prompt_1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

# Generating response back from gpt-3.5-turbo
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt_1}]
)

print(openai_response.choices[0].message.content)

In [ ]:
# diák adatainak feldolgozása - valóságban egy adatbázisba mentene
def submit_student_info(name, major, school, grades, club):
    print(f"Student Name: {name}")
    print(f"Major: {major}")
    print(f"School: {school}")
    print(f"Grades: {grades}")
    print(f"Club: {club}")

# fentebbi függvény LLM számára érthető leírása
student_custom_functions = [
    {
        'name': 'submit_student_info',
        'description': 'Submit student information for processing',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]



In [ ]:
from openai import OpenAI
import json
client = OpenAI()

# diákok "CV"-i
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

prompt1 = '''
I need to submit structured student information for processing to a database. This is the body of text to extract the information from:
'''

student_description = [prompt1+student_1_description,prompt1+student_2_description]
for i in student_description:
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # válasz JSON alapú értelmezése
    print(response.model_dump())
    # függvényhívás paramétereinek kinyerése
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)
    # függvény meghívása
    submit_student_info(**json_response)

